In [1]:
!conda create -y -n nilmtk python=3.9 pip
!source activate nilmtk && python -m ipykernel install --user --name nilmtk --display-name "Python (nilmtk)"

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.7.2

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda/envs/nilmtk

  added / updated specs:
    - pip
    - python=3.9


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    wheel-0.41.1               |     pyhd8ed1ab_0          56 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          56 KB

The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge None
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu None
  bzip2              conda-forge/linux-64::bzip2-1.0.8-h7f98852_4 None
  ca-certificates    conda-forge/linux-64::c

In [2]:
#!rm -r /home/jovyan/.local/share/jupyter/kernels/nilmtk
#!pip uninstall -y -q nilmtk nilm_metadata
!python3 -m pip install pandas numpy networkx scipy tables scikit-learn hmmlearn pyyaml matplotlib xgboost pyts
# Trick to install NILM regardless of its dependencies
!python3 -m pip install --no-deps git+https://github.com/nilmtk/nilmtk@master
!python3 -m pip install --no-deps git+https://github.com/nilmtk/nilm_metadata@master

  Using cached hmmlearn-0.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (160 kB)
  Using cached xgboost-1.7.6-py3-none-manylinux2014_x86_64.whl (200.3 MB)
  Using cached pyts-0.13.0-py3-none-any.whl (2.5 MB)
  Using cached scikit_learn-1.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.8 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.1.3
    Uninstalling scikit-learn-1.1.3:
      Successfully uninstalled scikit-learn-1.1.3
  Cloning https://github.com/nilmtk/nilmtk (to revision master) to /tmp/pip-req-build-9w17sw02
  Running command git clone --filter=blob:none --quiet https://github.com/nilmtk/nilmtk /tmp/pip-req-build-9w17sw02
  Resolved https://github.com/nilmtk/nilmtk to commit b2c514479cef478cab872cb635056da08d5352a1
  Preparing metadata (setup.py) ... done
  Created wheel for nilmtk: filename=nilmtk-0.4.0.dev1+git.b2c5144-py3-none-any.whl size=279177 sha256=0b9b0aee1b553d0b0cca12fd184489f687eb0befcc92e5

In [1]:
!which python
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from nilmtk import DataSet
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict
from joblib import Memory

from pprint import pprint

from nilmtk import DataSet
import multiprocessing as mp

from typing import Dict

/opt/conda/bin/python


In [3]:
memory = Memory(location='./cache')

In [4]:
path = "./datasets/NILM/iawe.h5"


@memory.cache
def load_dataset():
    try:
        dataset = DataSet(path)

        samples = []
        for building_idx, building in dataset.buildings.items():
            aggregate = next(building.elec.mains()[1].load()) + next(building.elec.mains()[1].load())
            for meter in building.elec.all_meters():

                data = list(meter.load())
                assert len(data) == 1

                assert len(meter.appliances) < 2

                # TODO: Poglej s kje jemlje sample Jakob.
                sample = (building_idx, list([a.type['type'] for a in meter.appliances]), data, meter.good_sections(), aggregate)

                samples.append(sample)
                
        return samples

    except Exception as e:
        dataset.store.close()
        raise e
        

dataset = load_dataset()

In [5]:

def data_preparation(dataset):
    
    out_data = {}
    X = defaultdict(lambda: [])
    for (idx, appliances, data, good_sections, aggregate) in dataset:
        name = "IAWE_{}".format(idx)
        if name not in out_data:
           out_data[name] = {
                "aggregate": aggregate
           }
        if not appliances:
            continue
        
        
        appliance = appliances[0]
        data = data[0]
        if appliance == "unknown":
            continue
        samples = [data[good.start:good.end] for good in good_sections]
        print(name, appliance, len(samples))
        # X[appliance].extend(samples)
        df = pd.concat(samples, axis=0)
        out_data[name][appliance] = pd.concat(samples, axis=0)
        
    # for appliance, samples in X.items():
    #     print(appliance, len(samples))
        
    return out_data


prepared_data  = data_preparation(dataset)


IAWE_1 fridge 84
IAWE_1 air conditioner 76
IAWE_1 air conditioner 106
IAWE_1 washing machine 27
IAWE_1 computer 208
IAWE_1 clothes iron 13
IAWE_1 television 210
IAWE_1 wet appliance 34
IAWE_1 motor 51


In [6]:
# watts to kWh given data frequency as a fraction of an hour (e.g. 0.5 for half-hourly data)
def watts2kwh(df, data_frequency):
    df = df/1000 * data_frequency
    return df
for house in prepared_data:
    for meter in prepared_data[house]:
        df = pd.DataFrame(prepared_data[house][meter]["power"]["active"])
        prepared_data[house][meter] = watts2kwh(df, 60/3600)




In [8]:
import pickle

# save to pickle
with open("./Energy_graph/data/processed/IAWE.pkl", 'wb') as handle:
            pickle.dump(dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [2]:
from IAWE_parser import parse_IAWE


In [3]:
parse_IAWE("../../datasets/NILM/","../data/test/IAWE.pkl")

IAWE_1 fridge 84
IAWE_1 air conditioner 76
IAWE_1 air conditioner 106
IAWE_1 washing machine 27
IAWE_1 computer 208
IAWE_1 clothes iron 13
IAWE_1 television 210
IAWE_1 wet appliance 34
IAWE_1 motor 51
Data successfully saved to  ../data/test/IAWE.pkl


In [5]:
pd.read_pickle("../data/test/REDD.pkl")

{'REDD_1': {'aggregate': physical_quantity             power
  type                       apparent
  2011-04-18 09:22:06-04:00  0.000000
  2011-04-18 09:22:13-04:00  0.000864
  2011-04-18 09:22:20-04:00  0.000870
  2011-04-18 09:22:27-04:00  0.000865
  2011-04-18 09:22:34-04:00  0.000867
  ...                             ...
  2011-05-24 15:56:34-04:00  0.000915
  2011-05-24 15:56:41-04:00  0.000912
  2011-05-24 15:56:48-04:00  0.000914
  2011-05-24 15:56:55-04:00  0.000915
  2011-05-24 15:57:02-04:00  0.000916
  
  [447729 rows x 1 columns],
  'fridge': physical_quantity             power
  type                         active
  2011-04-18 09:22:13-04:00  0.000012
  2011-04-18 09:22:20-04:00  0.000012
  2011-04-18 09:22:27-04:00  0.000012
  2011-04-18 09:22:34-04:00  0.000012
  2011-04-18 09:22:41-04:00  0.000012
  ...                             ...
  2011-05-24 15:56:06-04:00  0.000366
  2011-05-24 15:56:13-04:00  0.000369
  2011-05-24 15:56:20-04:00  0.000362
  2011-05-24 15:56:27-0